In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
from scipy.optimize import fsolve

In [ ]:
T = 20  # deg C
z = 8.75  # m
Le = 100  # m
D = 0.1  # m
rho = 998  # kg/m^3
mu = 1.01e-3  # Pa.s
g = 9.81  # m/s^2
Q = np.array([0.00, 0.01, 0.02, 0.03, 0.04, 0.05])  # m^3/s
H = np.array([20.63, 19.99, 17.80, 14.46, 10.33, 5.71])  # m
eta = np.array([0.00, 36.1, 56.0, 61.0, 54.1, 37.0]) / 100  # %

Ne = rho * g * Q * H

Q_without_zero = Q[1:]
Ne_without_zero = Ne[1:]
eta_without_zero = eta[1:]
N_without_zero = Ne_without_zero / eta_without_zero

In [ ]:
# plot H vs Q (H ~ Q^2)
Q_sq = Q**2
model_H_Q = linregress(Q_sq, H)
print(f'R^2 = {model_H_Q.rvalue**2:.4f}')
print(f'H = {model_H_Q.slope:.4f} Q^2 + {model_H_Q.intercept:.4f}')


def plot():
    _x = np.linspace(min(Q_sq), max(Q_sq), 100)
    _y = model_H_Q.slope * _x + model_H_Q.intercept

    plt.figure()
    plt.plot(Q_sq, H, 'o', label='Data')
    plt.plot(_x, _y, '--', label='Linear regression')
    plt.xlabel('$Q^2$ (m$^6$/s$^2$)')
    plt.ylabel('$H$ (m)')
    plt.title('$H$ vs $Q^2$')
    plt.legend()
    plt.grid()
    plt.show()
    return


plot()

In [ ]:
# plot eta vs Q (eta ~ Q^2)
eta_Q = eta_without_zero / Q_without_zero
model_eta_Q = linregress(Q_without_zero, eta_Q)
print(f'R^2 = {model_eta_Q.rvalue**2:.4f}')
print(f'eta = {model_eta_Q.slope:.4f} Q^2 + {model_eta_Q.intercept:.4f} Q')


def plot2():
    _x = np.linspace(min(Q_without_zero), max(Q_without_zero), 100)
    _y = model_eta_Q.slope * _x + model_eta_Q.intercept

    plt.figure()
    plt.plot(Q_without_zero, eta_Q, 'o', label='Data')
    plt.plot(_x, _y, '--', label='Linear regression')
    plt.xlabel('$Q$ (m$^3$/s)')
    plt.ylabel('$η$ / $Q$ (1 / m$^3$/s)')
    plt.title('$η$ / $Q$ vs $Q$')
    plt.legend()
    plt.grid()
    plt.show()
    return


plot2()

In [ ]:
def get_Re(Q_):
    return 4 * rho * Q_ / (np.pi * D * mu)


def get_lambda_f(Q_):
    _Re = get_Re(Q_)
    return 0.3164 / _Re**0.25


def get_H_f(Q_):
    _lambda_f = get_lambda_f(Q_)
    return 8 * _lambda_f * Le * Q_**2 / (np.pi**2 * D**5 * g)


def get_H_v(Q_):
    return (4 * Q_ / (np.pi * D**2)) ** 2 / (2 * g)


def get_H_required(Q_):
    return get_H_f(Q_) + get_H_v(Q_) + z


def H_provided(Q_):
    return model_H_Q.slope * Q_**2 + model_H_Q.intercept


def func(Q_):
    return H_provided(Q_) - get_H_required(Q_)


def get_eta(Q_):
    return model_eta_Q.slope * Q_**2 + model_eta_Q.intercept * Q_


solutions = fsolve(func, 0.01)

Q_sol = solutions[0]
Re_sol = get_Re(Q_sol)
H_sol = H_provided(Q_sol)
eta_sol = get_eta(Q_sol)

print(f'Q = {Q_sol:.4f} m^3/s')
print(f'Re = {Re_sol:.2e}')
print(f'H = {H_sol:.2f} m')
print(f'η = {eta_sol * 100:.2f} %')